In [3]:
from mbert_trainer import Classifier
from transformers import BertTokenizer
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from tqdm.notebook import tqdm

In [4]:
test = pd.read_csv('test.csv', encoding='utf-8')

In [8]:
checkpoint = 'bert-base-cased'

In [9]:
mbert = Classifier(checkpoint)
mbert.load_state_dict(torch.load('saved_models/bert-base-cased-68.75.pt'))

<All keys matched successfully>

In [10]:
sentences = test.sentence.values
NUM_EXAMPLES = len(sentences)

In [11]:
tokenizer = BertTokenizer.from_pretrained(checkpoint)
input_ids = []
attention_mask = []
for i in tqdm(range(0, NUM_EXAMPLES, 1000)):
    encoded_content = tokenizer.batch_encode_plus(
        sentences[i:i+1000], 
        add_special_tokens=True,
        max_length=200,
        truncation=True,
        padding='max_length',
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='pt',
        return_dict=False
    )
    input_ids.append(encoded_content['input_ids'])
    attention_mask.append(encoded_content['attention_mask'])

  0%|          | 0/6 [00:00<?, ?it/s]

In [12]:
input_ids = torch.cat(input_ids, dim=0)
attention_mask = torch.cat(attention_mask, dim=0)

In [13]:
predictions = []
batch_size = 4
for i in tqdm(range(0, NUM_EXAMPLES, batch_size)):
    size = input_ids[i:i+batch_size, :].shape[0]
    pred = mbert.predict_prob(
        input_ids[i:i+batch_size, :].reshape(size, 200),
        attention_mask[i:i+batch_size, :].reshape(size, 200)
    )
    predictions.extend(pred)

  0%|          | 0/93 [00:00<?, ?it/s]